# Binary Sentiment Analysis of French Movie Reviews

### Objectives
1. Text cleaning
2. Text preprocessing for custom embedding Neural Network
3. Train RNN model for sentiment analysis

⚠️ This notebook will be your final deliverable. 
- Make sure it can run "restart and run all"
- Delete useless code cells
- Do not "clear output"

# 0. Load data

Our dataset contains 30,000 french reviews of movies, along with the binary class 1 (positive) or 0 (negative) score

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns

In [2]:
# We load the dataset for you
data = pd.read_csv('https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies.csv')
data

,review,polarity
0,Ça commence à devenir énervant d'avoir l'impre...,0
1,"J'ai aimé ce film, si il ressemble a un docume...",1
2,Une grosse merde ce haneke ce faire produire p...,0
3,"Beau mélodrame magnifiquement photographié, ""V...",1
4,A la poursuite du diamant vers est un film pro...,1
...,...,...
29946,Le meilleur film de super-héros derrière le ba...,1
29947,Un drame qui est d'une efficacité remarquable....,1
29948,"Une daube hollywoodienne de plus, aucun intérê...",0
29949,Et voilà un nouveau biopic sur la star du X Li...,0


In [3]:
# We create features
y = data.polarity
X = data.review

# We analyse class balance
print(pd.value_counts(y))

1    15051
0    14900
Name: polarity, dtype: int64


In [4]:
data.describe()

,polarity
count,29951.000000
mean,0.502521
std,0.500002
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [5]:
# We check various reviews
print(f'polarity: {y[0]} \n')
print(X[0])

polarity: 0 

Ça commence à devenir énervant d'avoir l'impression de voir et revoir le même genre de film à savoir : la comédie romantique, surement le genre le plus prolifique de le production française actuelle. Le problème c'est que l'on a souvent affaire à des niaiseries de faible niveau comme celui ci. Avec un scenario ultra balisé et conventionnel, c'est à se demander comment ça peut passer les portes d'un producteur. Bref cette sempiternel histoire d'un homme mentant au nom de l'amour pour reconquérir une femme et qui à la fin se prend son mensonge en pleine figure est d'une originalité affligeante, et ce n'est pas la présence au casting de l'ex miss météo Charlotte Le Bon qui rêve surement d'avoir la même carrière que Louise Bourgoin qui change la donne.


# 1. Clean Text

❓ We need to give a _quick & dirty_ cleaning to all the sentences in the dataset. Create a variable `X_clean` of similar shape, but with the following cleaning:
- Replace french accents by their non-accentuated equivalent using the [unidecode.unidecode()](https://pypi.org/project/Unidecode/) method
- Reduce all uppercases to lowercases
- Remove any characters outside of a-z, for instance using `string.isalpha()`

😌 You will be given the solution `X_clean` in the next question to make sure you can complete the challenge

In [6]:
### YOUR CODE
import unidecode
def accents(sentence):
    no_accent = unidecode.unidecode(sentence)
    return no_accent

In [7]:
### Remove accents
X_clean = X.copy()
X_clean = X_clean.map(accents)

In [8]:
### Lowercases
X_clean = X_clean.map(lambda text: text.lower())

In [9]:
### Remove non alpha char
import re
def remove_nonalpha(text):
    text = ''.join(c for c in text if c.isalpha() or c == ' ')
    return re.sub(" +", " ", text)

In [10]:
X_clean = X_clean.map(remove_nonalpha)

In [11]:
X_clean[0]

'ca commence a devenir enervant davoir limpression de voir et revoir le meme genre de film a savoir la comedie romantique surement le genre le plus prolifique de le production francaise actuelle le probleme cest que lon a souvent affaire a des niaiseries de faible niveau comme celui ci avec un scenario ultra balise et conventionnel cest a se demander comment ca peut passer les portes dun producteur bref cette sempiternel histoire dun homme mentant au nom de lamour pour reconquerir une femme et qui a la fin se prend son mensonge en pleine figure est dune originalite affligeante et ce nest pas la presence au casting de lex miss meteo charlotte le bon qui reve surement davoir la meme carriere que louise bourgoin qui change la donne'

In [12]:
from nbresult import ChallengeResult

result = ChallengeResult('C14',
    shape = X_clean.shape,
    first_sentence = X_clean[0]
)
result.write()

# 2. Preprocess data

Now that we have clean sentences, we need to convert each one into a list of integers of fixed size
- For example, the sentence: `"this was good"` should become something like `array([1, 3, 18, 0, 0, 0, ...0], dtype=int32)` where each integer match to a each _unique_ word in your corpus of sentences.

❓ Create a numpy ndarray `X_input` of shape (29951, 100) that will be the direct input to your Neutral Network. 

- 29951 represents the number of reviews in the dataset `X_clean`
- 100 represents the maximum number of words to keep for each movie review.
- It must contain only numerical values, without any `NaN`
- In the process, compute and save the number of _unique_ words in your cleaned corpus under `vocab_size` variable

👉 First, you **must** start back from the clean solution below (14Mo)

In [13]:
X_clean = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies_X_clean.csv")['review']
X_clean

0        ca commence a devenir enervant de voir et revo...
1        aime ce film si il ressemble a un documentaire...
2        une grosse merde ce haneke ce faire produire p...
3        beau melodrame magnifiquement photographie ver...
4        a la poursuite du diamant vers est un film pro...
                               ...                        
29946    le meilleur film de derriere le batman de nola...
29947    un drame qui est efficacite remarquable un fil...
29948    une daube hollywoodienne de plus aucun interet...
29949    et voila un nouveau biopic sur la star du x li...
29950    un film qui fait vieux avec des acteurs pas to...
Name: review, Length: 29951, dtype: object

In [21]:
### YOUR CODE
vec = CountVectorizer(max_features=100)
X_input = vec.fit_transform(X_clean)
X_input = X_input.toarray()

In [22]:
X_input

array([[0, 0, 2, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 2, ..., 0, 0, 0]])

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit_transform(X_clean)
vectorizer.get_feature_names()

# Verify Vocab Size
vocab_size = pd.Series(vectorizer.get_feature_names()).nunique()
vocab_size

62353

In [24]:
from nbresult import ChallengeResult

result = ChallengeResult('C1415',
    type_X = type(X_input),
    shape = X_input.shape, 
    input_1 = X_input[1], 
)
result.write()

# 3. Neural Network

❓Create and fit a Neural Netork that takes `X_input` and `y` as input, to binary classify each sentence's sentiment

- You cannot use transfer learning or other pre-existing Word2Vec models
- You must use a "recurrent" architecture to _capture_ a notion of order in the sentences' words
- The performance metrics for this task is "accuracy"
- Store your model in a variable `model` 
- Store the result your `model.fit()` in a variable `history`. 
- ⚠️ `history.history` must comprises a measure of the `val_accuracy` at each epoch.
- You don't need to cross-validate your model

😌 Don't worry, you will not be judged on your computer power: You should be able to reach accuracy significantly better than baseline in less than 3 minutes even without GPUs.

👉 But first, you **must** start back from the solution below (70Mo)

In [25]:
url = 'https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies_X_input.csv'
X_input = np.genfromtxt(url, delimiter=',', dtype='int32')

In [29]:
X_input.max()

62379

In [39]:
## YOUR CODE
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(layers.Embedding(input_dim=62380, output_dim=100))  # We have 62379 words in the vocabulary, 
                                                              # and each word is represnted by a vector of size 100
model.add(layers.LSTM(25, activation='tanh'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [40]:
es = EarlyStopping(patience=2, restore_best_weights=True)
history = model.fit(X_input, y,
                    batch_size = 32,
                    epochs=100,
                    validation_split=0.3,
                    callbacks=[es]
                    )

Epoch 1/100
656/656 [==============================] - 50s 73ms/step - loss: 0.5750 - accuracy: 0.6531 - val_loss: 0.2828 - val_accuracy: 0.8862
Epoch 2/100
656/656 [==============================] - 47s 72ms/step - loss: 0.2429 - accuracy: 0.9045 - val_loss: 0.3791 - val_accuracy: 0.8759
Epoch 3/100
656/656 [==============================] - 46s 70ms/step - loss: 0.1892 - accuracy: 0.9285 - val_loss: 0.2635 - val_accuracy: 0.8941
Epoch 4/100
656/656 [==============================] - 46s 70ms/step - loss: 0.1590 - accuracy: 0.9446 - val_loss: 0.3824 - val_accuracy: 0.8561
Epoch 5/100
656/656 [==============================] - 45s 69ms/step - loss: 0.1431 - accuracy: 0.9516 - val_loss: 0.3076 - val_accuracy: 0.8912


In [44]:
from statistics import mean
mean(history.history['val_accuracy'])

0.8806810498237609

In [45]:
from nbresult import ChallengeResult
result = ChallengeResult('C1517',
                         history=history.history)
result.write()